In [1]:
x1=[1,2,3]
x2=[4,5,6]

X=[x1,x2,x1,x2]

In [2]:
for i in X:
    print(i)
    i[1]=0

[1, 2, 3]
[4, 5, 6]
[1, 0, 3]
[4, 0, 6]


In [2]:
x=[1,2,3]
y=[4,5,6]
z=[]
z+=x+y
z

[1, 2, 3, 4, 5, 6]

In [18]:
import numpy as np
n=10
A=np.arange(n**2).reshape((n,n))
B=np.dot(np.transpose(A),A)
np.linalg.pinv(B)

array([[ 1.24782369e-03,  9.78071625e-04,  7.08319559e-04,
         4.38567493e-04,  1.68815427e-04, -1.00936639e-04,
        -3.70688705e-04, -6.40440771e-04, -9.10192837e-04,
        -1.17994490e-03],
       [ 9.78071625e-04,  7.66648301e-04,  5.55224977e-04,
         3.43801653e-04,  1.32378329e-04, -7.90449954e-05,
        -2.90468320e-04, -5.01891644e-04, -7.13314968e-04,
        -9.24738292e-04],
       [ 7.08319559e-04,  5.55224977e-04,  4.02130395e-04,
         2.49035813e-04,  9.59412305e-05, -5.71533517e-05,
        -2.10247934e-04, -3.63342516e-04, -5.16437098e-04,
        -6.69531680e-04],
       [ 4.38567493e-04,  3.43801653e-04,  2.49035813e-04,
         1.54269972e-04,  5.95041322e-05, -3.52617080e-05,
        -1.30027548e-04, -2.24793388e-04, -3.19559229e-04,
        -4.14325069e-04],
       [ 1.68815427e-04,  1.32378329e-04,  9.59412305e-05,
         5.95041322e-05,  2.30670340e-05, -1.33700643e-05,
        -4.98071625e-05, -8.62442608e-05, -1.22681359e-04,
        -1.

In [23]:
test=[[1,2,3],[4,5,6],[7,8,9]]
l=test[0]
l.remove(2)
test

[[1, 3], [4, 5, 6], [7, 8, 9]]